In [1]:
# Cell 1: Import all necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import i0, modstruve, jv, kn  # Modified Bessel functions
from scipy.integrate import quad
from scipy.stats import rayleigh, norm
import matplotlib.patches as patches
from IPython.display import display, Math, Latex

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
%config InlineBackend.figure_format = 'retina'

# Cell 2: Title and Introduction
# Non-Coherent Detection in Digital Communications

## Introduction

In an ideal **coherent** communication system, the receiver possesses perfect knowledge of the carrier wave's phase, frequency, and timing. This allows it to construct a perfectly matched filter, projecting the received signal onto the exact basis functions used at the transmitter. This maximizes the signal-to-noise ratio (SNR) and minimizes the probability of error.

However, in many practical scenarios (e.g., fast-varying mobile channels, low-cost oscillators, high Doppler shifts), it is prohibitively complex or impossible for the receiver to acquire and track the carrier phase. **Non-coherent detection** is a technique that allows for the detection of digital signals *without* knowledge of the carrier phase. While it is sub-optimal compared to coherent detection, its simplicity and robustness make it indispensable for systems like amplitude shift keying (ASK) and frequency shift keying (FSK).

This notebook will develop the statistical framework for non-coherent detection, focusing on the envelope of the received signal as the key decision metric.

In [ ]:
# Cell 3: Conditional PDFs - Mathematical Setup
display(Math(r'\text{The received signal after an AWGN channel is } r(t) = s_i(t) + n(t)'))
display(Math(r'\text{We process } r(t) \text{ through quadrature correlators:}'))
display(Math(r'r_I = \int_0^{T_b} r(t) \phi_1(t)  dt'))
display(Math(r'r_Q = \int_0^{T_b} r(t) \phi_2(t)  dt'))
display(Math(r'\text{where } \phi_1(t) = \sqrt{\frac{2}{T_b}}\cos(2\pi f_c t), \quad \phi_2(t) = -\sqrt{\frac{2}{T_b}}\sin(2\pi f_c t)'))

display(Math(r'\text{Under hypothesis } H_0 \text{ (signal absent):}'))
display(Math(r'p(r_I, r_Q | H_0) = \frac{1}{2\pi\sigma^2} \exp\left( -\frac{r_I^2 + r_Q^2}{2\sigma^2} \right)'))

display(Math(r'\text{Under hypothesis } H_1 \text{ (signal present with unknown phase } \theta):'))
display(Math(r'p(r_I, r_Q | H_1) = \frac{1}{2\pi} \int_{0}^{2\pi} \frac{1}{2\pi\sigma^2} \exp\left( -\frac{(r_I - A\cos\theta)^2 + (r_Q - A\sin\theta)^2}{2\sigma^2} \right) d\theta'))

In [ ]:
# Cell 4: Signal Space Visualization
# Create a figure to show the signal space concept
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Coherent detection (known phase)
ax[0].plot(0, 0, 'bo', markersize=10, label='$H_0$')
ax[0].plot(1, 0, 'ro', markersize=10, label='$H_1$')
ax[0].set_xlim(-0.5, 1.5)
ax[0].set_ylim(-0.5, 0.5)
ax[0].set_xlabel('In-phase component')
ax[0].set_ylabel('Quadrature component')
ax[0].set_title('Coherent Detection: Known Phase')
ax[0].grid(True)
ax[0].legend()
ax[0].set_aspect('equal')

# Non-coherent detection (unknown phase)
theta = np.linspace(0, 2*np.pi, 100)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax[1].plot(0, 0, 'bo', markersize=10, label='$H_0$')
ax[1].plot(circle_x, circle_y, 'r-', alpha=0.5)
ax[1].plot(circle_x, circle_y, 'ro', markersize=3, alpha=0.5, label='Possible $H_1$ locations')
ax[1].set_xlim(-1.5, 1.5)
ax[1].set_ylim(-1.5, 1.5)
ax[1].set_xlabel('In-phase component')
ax[1].set_ylabel('Quadrature component')
ax[1].set_title('Non-Coherent Detection: Unknown Phase')
ax[1].grid(True)
ax[1].legend()
ax[1].set_aspect('equal')

plt.tight_layout()
plt.show()

# Cell 5: Decision Metric Explanation

## Decision Metric and Signal Space

Working with the Cartesian coordinates $(r_I, r_Q)$ is complicated due to the dependence on $\theta$. A crucial insight is to transition from Cartesian to polar coordinates.

Define the **envelope** $R$ and the **phase** $\Psi$ of the received vector:
$$
R = \sqrt{r_I^2 + r_Q^2}, \quad \Psi = \tan^{-1}\left( \frac{r_Q}{r_I} \right)
$$

- **Signal Space Representation:** In non-coherent detection, the transmitted symbol $H_1$ is no longer represented by a single point $(A, 0)$ in the signal space. Because the phase $\theta$ is unknown, the point can lie *anywhere* on a circle of radius $A$ centered at the origin. The symbol $H_0$ is still represented by the origin $(0, 0)$.

- **The Decision Metric:** The optimal decision rule simplifies to comparing the envelope $R$ to a threshold $\gamma_0$. The receiver decides:
  $$
  \begin{aligned}
  \text{If } R \geq \gamma_0 &\quad \Rightarrow \quad \text{Decide } \hat{H}_1 \\
  \text{If } R < \gamma_0  &\quad \Rightarrow \quad \text{Decide } \hat{H}_0
  \end{aligned}
  $$

Intuitively, if the received signal vector has a large magnitude (envelope), it is more likely that a signal was present, regardless of its specific phase. If the magnitude is small, it is more likely to be just noise.

In [ ]:
# Cell 6: Rayleigh and Rician Distribution Visualization

sigma = 1.0    # Noise standard deviation
s = 2.0        # Amplitude of dominant signal component
K = s**2 / (2 * sigma**2) # Calculate K-factor

r = np.linspace(0, 5, 1000)  # Range of envelope values r

# 1. Rayleigh PDF (H0: Noise only)
rayleigh_pdf = (r / sigma**2) * np.exp(-r**2 / (2 * sigma**2))

# 2. Rician PDF (H1: Signal + Noise)
rician_pdf = (r / sigma**2) * np.exp(-(r**2 + s**2) / (2 * sigma**2)) * i0((r * s) / sigma**2)

# 3. (For comparison) Gaussian PDF that the Rician approaches for large K
mu_gaussian = s
sigma_gaussian = sigma
gaussian_pdf = norm.pdf(r, mu_gaussian, sigma_gaussian)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(r, rayleigh_pdf, 'b-', linewidth=2, label=f'Rayleigh ($\sigma$={sigma})')
plt.plot(r, rician_pdf, 'r--', linewidth=2, label=f'Rician (s={s}, $\sigma$={sigma}, K={K:.1f})')
plt.plot(r, gaussian_pdf, 'g:', linewidth=2, label=f'Gaussian ($\mu$={s}, $\sigma$={sigma})')

plt.title('Probability Density Functions: Rayleigh vs. Rician')
plt.xlabel('Envelope Magnitude, $r$')
plt.ylabel('Probability Density, $p_R(r)$')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(bottom=0)
plt.show()

display(Math(r'\text{Rayleigh PDF: } p_R(r | H_0) = \frac{r}{\sigma^2} \exp\left( -\frac{r^2}{2\sigma^2} \right)'))
display(Math(r'\text{Rician PDF: } p_R(r | H_1) = \frac{r}{\sigma^2} \exp\left( -\frac{r^2 + s^2}{2\sigma^2} \right) I_0\left( \frac{r s}{\sigma^2} \right)'))

In [ ]:
# Cell 7: BER Calculation and Comparison
# Define the range of SNR values
EbN0_dB = np.linspace(0, 20, 200)  # SNR in dB
EbN0_linear = 10**(EbN0_dB/10)     # Convert to linear scale

# Coherent BPSK BER
coherent_ber_approx = 0.5 * np.exp(-EbN0_linear)  # Approximation for high SNR

# Non-coherent BFSK BER
noncoherent_ber = 0.5 * np.exp(-EbN0_linear / 2)

# Plot the results
plt.figure(figsize=(10, 6))
plt.semilogy(EbN0_dB, coherent_ber_approx, 'b--', linewidth=1, alpha=0.7, label='Coherent BPSK (Approx)')
plt.semilogy(EbN0_dB, noncoherent_ber, 'r-', linewidth=2, label='Non-coherent BFSK')

plt.xlabel('$E_b/N_0$ (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('BER Comparison: Coherent vs. Non-coherent Detection')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.ylim(1e-10, 1)
plt.xlim(0, 20)
plt.show()

display(Math(r'\text{Coherent BPSK: } P_e^{\text{Coherent}} \approx Q\left( \sqrt{\frac{2E_b}{N_0}} \right) \approx \frac{1}{2} e^{-E_b/N_0}'))
display(Math(r'\text{Non-coherent BFSK: } P_e^{\text{Non-coherent}} = \frac{1}{2} e^{-E_b/(2N_0)}'))
display(Math(r'\text{Non-coherent detection requires } \approx \text{1-1.5 dB more } E_b/N_0 \text{ for the same BER}'))

In [ ]:
# Cell 8: Simulation of Non-coherent ASK Detection
def noncoherent_ask_simulation(snr_db, num_bits=10000):
    """
    Simulate non-coherent ASK detection
    """
    # Convert SNR from dB to linear scale
    snr_linear = 10**(snr_db/10)
    
    # For ASK, symbol energy Es = Eb (on average, since half the bits are 0)
    # Noise variance per dimension: σ² = N₀/2
    # For unit energy signal, Eb = 1, so N₀ = 1/SNR
    sigma = np.sqrt(1/(2*snr_linear))
    
    # Generate random bits (0s and 1s)
    tx_bits = np.random.randint(0, 2, num_bits)
    
    # Create transmitted signal (Amplitude A for 1, 0 for 0)
    A = 1.0  # Signal amplitude
    tx_signal = A * tx_bits
    
    # Add noise (I and Q components)
    noise_i = np.random.normal(0, sigma, num_bits)
    noise_q = np.random.normal(0, sigma, num_bits)
    
    # Received signal (with unknown phase)
    # For simplicity, we assume phase is random but constant per symbol
    theta = np.random.uniform(0, 2*np.pi, num_bits)
    rx_i = tx_signal * np.cos(theta) + noise_i
    rx_q = tx_signal * np.sin(theta) + noise_q
    
    # Envelope detection
    envelope = np.sqrt(rx_i**2 + rx_q**2)
    
    # Optimal threshold for minimum P_e is approximately A/2
    threshold = A/2
    
    # Decision
    rx_bits = (envelope > threshold).astype(int)
    
    # Calculate bit error rate
    ber = np.sum(rx_bits != tx_bits) / num_bits
    
    return ber

# Run simulation for multiple SNR values
snr_values_db = np.array([0, 2, 4, 6, 8, 10, 12, 14])
simulated_ber = []

for snr in snr_values_db:
    ber = noncoherent_ask_simulation(snr, 50000)
    simulated_ber.append(ber)

# Theoretical BER for non-coherent ASK
theoretical_ber = 0.5 * np.exp(-10**(snr_values_db/10)/2)

# Plot results
plt.figure(figsize=(10, 6))
plt.semilogy(snr_values_db, theoretical_ber, 'r-', linewidth=2, label='Theoretical')
plt.semilogy(snr_values_db, simulated_ber, 'bo', markersize=6, label='Simulated')
plt.xlabel('$E_b/N_0$ (dB)')
plt.ylabel('Bit Error Rate (BER)')
plt.title('Non-coherent ASK: Theoretical vs. Simulated BER')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.ylim(1e-4, 1)
plt.show()

# Cell 9: Conclusion

## Summary and Conclusion

Non-coherent detection provides a practical alternative to coherent detection when phase synchronization is difficult or expensive to implement. The key points are:

1.  **Unknown Phase Handling:** By using envelope detection instead of phase-sensitive correlation, non-coherent receivers can demodulate signals without knowing the carrier phase.

2.  **Statistical Framework:** The received signal envelope follows:
    - **Rayleigh distribution** when only noise is present
    - **Rician distribution** when signal plus noise is present

3.  **Performance Trade-off:** Non-coherent detection typically requires about 1-1.5 dB higher $E_b/N_0$ compared to coherent detection to achieve the same bit error rate. This is the price paid for not knowing the carrier phase.

4.  **Applications:** Non-coherent detection is widely used in:
    - Low-cost wireless systems
    - Frequency-hopping spread spectrum
    - Systems with rapid phase variations (e.g., mobile communications)
    - Simple receivers where complexity must be minimized

The mathematical framework developed in this notebook provides the foundation for understanding and analyzing non-coherent communication systems.